Assumption:
- GB by 2035, 50GW of Offshore Wind, and 70MW of Solar
- 14GW of Onshore Wind from today remains
- Does not consider Nuclear in the model
- Use historical half-hourly load (2015-2019)
- Use historical hourly wind and solar cf (2015-2019)

Goal:
- What is the best investment decision that can be made in order to support the future VRE-based power system, given a budget?

Options:
- Budget: 5b, 10b
- Generation technologies: gt no storage, gt and only pumped hydro, gt with all storage options (Table 1)
- Reserve requirements: No reserve, standard reserve, vre reserve (Table 2)


In [64]:
using CSV, DataFrames, JuMP, HiGHS, SCIP, Clp

storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
generatorinfo = CSV.read(joinpath("cleaned","generatorinfo.csv"), DataFrame)
vreinfo = CSV.read(joinpath("cleaned","vreinfo.csv"), DataFrame)
halfhourlydemand = CSV.read(joinpath("cleaned","halfhourlydemand.csv"), DataFrame)
halfhourlyvrecf = CSV.read(joinpath("cleaned","halfhourlyvrecf.csv"), DataFrame)
halfhourlydemand = halfhourlydemand[30000:30000+10000,:]

Row,hh,load
,Int64,Int64
1,30000,22635
2,30001,21974
3,30002,21685
4,30003,21147
5,30004,20772
6,30005,20869
7,30006,20808
8,30007,20619
9,30008,20521


In [65]:
function planning_model(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, curtailpenalty, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)
    # set_optimizer_attribute(model, "mip_rel_gap", epsilon)
    # set_optimizer(model, SCIP.Optimizer)
    # set_attribute(model, "display/verblevel", 0)
    # set_attribute(model, "limits/gap", epsilon)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        R_GEN[G,T] >= 0
        SOC[S,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        R_DISCHARGE[S,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
        # CAPG_COMMIT[G,T] >= 0
        # COMMIT[G,T], Bin
        # START[G,T], Bin
        # SHUT[G,T], Bin
        # CSTATE[S,T], Bin
        # DSTATE[S,T], Bin
        # CAPS_CSTATE[S,T] >= 0
        # CAPS_DSTATE[S,T] >= 0
        # STATE[S,T], Bin
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, StorPowInvCost, sum(0.85e3 * storageinfo[storageinfo.id.==s,:powerCost][1] * CAPS[s] for s in S))
    @expression(model, StorEnInvCost, sum(0.85e3 * storageinfo[storageinfo.id.==s,:energyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GenInvCost + StorPowInvCost + StorEnInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, ReserveProvisionCost,
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * R_GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * curtailpenalty * CURTAIL[v,t] for v in V for t in T))
    @expression(model, PenaltyChargeDischarge, 10 * sum(CHARGE[g,t] + DISCHARGE[g,t] for g in G for t in T))
    @objective(model, Min, EnergyProvisionCost + ReserveProvisionCost + UnservedEnergyCost + CurtailedVreCost + PenaltyChargeDischarge)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) +
        sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    # @constraint(model, c02[g in G, t in T], CAPG_COMMIT[g,t] <= bigm * COMMIT[g,t])
    # @constraint(model, c03[g in G, t in T], CAPG_COMMIT[g,t] <= CAPG[g])
    # @constraint(model, c04[g in G, t in T], CAPG_COMMIT[g,t] >= CAPG[g] - (1- COMMIT[g,t]) * bigm)
    # @constraint(model, c05[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG_COMMIT[g,t])
    # @constraint(model, c06[g in G, t in T], 
    #     GEN[g,t] >= generatorinfo[generatorinfo.id.==g,:minPrate][1] * CAPG_COMMIT[g,t])
    # @constraint(model, c46[g in G, t in T], 
    #     GEN[g,t] + R_GEN[g,t] >= generatorinfo[generatorinfo.id.==g,:minPrate][1] * CAPG_COMMIT[g,t])
    @constraint(model, c90[g in G, t in T], GEN[g,t] + R_GEN[g,t] <= CAPG[g])
    # @constraint(model, c91[g in G, t in T], GEN[g,t] >= generatorinfo[generatorinfo.id.==g,:minPrate][1] * CAPG[g])

    # # GENERATOR START CONSTRAINT
    # @constraint(model, c07[g in G, t in T], 
    #     COMMIT[g,t] >= sum(START[g,i] for i in max(1,t-generatorinfo[generatorinfo.id.==g,:minupTime][1]):t))
    
    # # GENERATOR SHUT CONSTRAINT
    # @constraint(model, c08[g in G, t in T], 
    #     1 - COMMIT[g,t] >= sum(SHUT[g,i] for i in max(1,t-generatorinfo[generatorinfo.id.==g,:mindownTime][1]):t))
    
    # # GENERATOR COMMIT CONSTRAINT
    # @constraint(model, c09[g in G, t in T1], COMMIT[g,t] - COMMIT[g,t-1] == START[g,t] - SHUT[g,t])
    # @constraint(model, c10[g in G, t in T], COMMIT[g,t] <= sum(START[g,i] for i in minimum(T):t))

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])
    
    # STORAGE CHARGE LIMIT CONSTRAINT
    # @constraint(model, c25[s in S, t in T], CAPS_CSTATE[s,t] <= bigm * CSTATE[s,t])
    # @constraint(model, c26[s in S, t in T], CAPS_CSTATE[s,t] <= CAPS[s])
    # @constraint(model, c27[s in S, t in T], CAPS_CSTATE[s,t] >= CAPS[s] - (1- CSTATE[s,t]) * bigm)
    # @constraint(model, c28[s in S, t in T], CHARGE[s,t] <= CAPS_CSTATE[s,t])
    # @constraint(model, c29[s in S], CSTATE[s,1] == 0)
    @constraint(model, c92[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # STORAGE DISCHARGE LIMIT CONSTRAINT
    # @constraint(model, c30[s in S, t in T], CAPS_DSTATE[s,t] <= bigm * DSTATE[s,t])
    # @constraint(model, c31[s in S, t in T], CAPS_DSTATE[s,t] <= CAPS[s])
    # @constraint(model, c32[s in S, t in T], CAPS_DSTATE[s,t] >= CAPS[s] - (1- DSTATE[s,t]) * bigm)
    # @constraint(model, c33[s in S, t in T], DISCHARGE[s,t] + R_DISCHARGE[s,t] <= CAPS_DSTATE[s,t])
    # @constraint(model, c34[s in S], DSTATE[s,1] == 0)
    @constraint(model, c93[s in S, t in T], DISCHARGE[s,t] + R_DISCHARGE[s,t] <= CAPS[s])

    # STORAGE STATE CONSTRAINT
    # @constraint(model, c36[s in S, t in T], CSTATE[s,t] + DSTATE[s,t] <= 1)
    
    # STORAGE SOC LIMIT CONSTRAINT
    @constraint(model, c19[s in S, t in T], SOC[s,t] <= SOCM[s])
    @constraint(model, c20[s in S], SOC[s,minimum(T)] == 0.5 * SOCM[s])
    @constraint(model, c21[s in S], SOC[s,maximum(T)] == 0.5 * SOCM[s])

    # STORAGE SOC UPDATE CONSTRAINT
    @constraint(model, c22[s in S, t in T1], SOC[s,t] == SOC[s,t-1] + 
        (CHARGE[s,t] * storageinfo[storageinfo.id.==s,:eff][1]) - 
        (DISCHARGE[s,t] / storageinfo[storageinfo.id.==s,:eff][1]))
    @constraint(model, c23[s in S, t in T], R_DISCHARGE[s,t] <= SOC[s,t])

    # VRE RESERVE REQUIREMENT CONSTRAINT
    @constraint(model, c24[t in T], 
        sum(R_GEN[g,t] for g in G) + sum(R_DISCHARGE[s,t] for s in S) >= 
        0.1 * halfhourlydemand[halfhourlydemand.hh.==t,:load][1])

    # # CHARGE DISCHARGE
    # @constraint(model, c43[s in S, t in T],
    #     CHARGE[s,t] - STATE[s,t] * bigm <= 0)
    # @constraint(model, c44[s in S, t in T],
    #     DISCHARGE[s,t] + STATE[s,t] * bigm <= bigm)

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        R_GEN = value.(R_GEN).data,
        SOC = value.(SOC).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        R_DISCHARGE = value.(R_DISCHARGE).data,
        NET_DISCHARGE = value.(DISCHARGE).data .- value.(CHARGE).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        # CAPG_COMMIT = value.(CAPG_COMMIT).data,
        # COMMIT = value.(COMMIT).data,
        # START = value.(START).data,
        # SHUT = value.(SHUT).data,
        # CSTATE = value.(CSTATE).data,
        # DSTATE = value.(DSTATE).data,
        GenInvCost = value(GenInvCost),
        StorPowInvCost = value(StorPowInvCost),
        StorEnInvCost = value(StorEnInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        ReserveProvisionCost = value(ReserveProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

planning_model (generic function with 1 method)

In [66]:
solutions = planning_model(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=100e3, curtailpenalty=0.1, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
780033 rows, 634167 cols, 2494257 nonzeros
670764 rows, 524898 cols, 2967376 nonzeros
670764 rows, 510795 cols, 2953273 nonzeros
Presolve : Reductions: rows 670764(-139328); columns 510795(-129299); elements 2953273(+423002)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 20002(1.0943e+10) 3s
       1431     1.5754095421e+04 Pr: 70842(8.58589e+12); Du: 0(3.0512e-07) 9s
       2557     3.9431563269e+04 Pr: 61531(1.56101e+12); Du: 0(2.88783e-07) 17s
       3345     5.6145406874e+04 Pr: 151569(9.64703e+12); Du: 0(1.99059e-07) 23s
       4105     7.1239034053e+04 Pr: 102454(6.34291e+12); Du: 0(3.65389e-07) 29s
       4906     8.7974981592e+04 Pr: 102609(6.43339e+12); Du: 0(4.81091e-07) 35s
       5733     1.0328916101e+05 Pr: 92731(1.56815e+13); Du: 0(4.06901e-07) 42s
       6485     1.165027385

(CAPG = [6661.025437683311, 34174.8], CAPS = [-0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 7295.462374063987, 2254.2121882527, -0.0, -0.0], SOCM = [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 51390.7740549433, 59881.881095206365, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1458.3878117472987 0.0], R_GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0; … ; -0.0 -0.0 … 0.0 0.0; -0.0 -0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 -0.0], R_DISCHARGE = [0.0 0.0 … 0.0 0.0; -0.0 0.0 … 0.0 0.0; … ; -0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 -0.0], CURTAIL = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 4511.725437683307 3836.7254376833143 … 0.0 0.0], NSE = [0.0, 0.0, 0.0, 0.0, 0.

In [43]:
solutions.CURTAIL

3×1001 Matrix{Float64}:
    0.0       0.0       0.0      0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
    0.0    1151.64   1792.4      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 9592.33  32370.0   32370.0  22209.4     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [44]:
solutions.GEN

2×1001 Matrix{Float64}:
   -0.0    -0.0    -0.0    -0.0  …     -0.0     -0.0     -0.0     -0.0
 9021.1  9021.1  9021.1  9021.1     14674.3  22552.8  22552.8  19286.9

In [45]:
solutions.R_GEN

2×1001 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [46]:
solutions.CAPG

2-element Vector{Float64}:
    -0.0
 22552.761892771745

In [47]:
solutions.DISCHARGE

14×1001 Matrix{Float64}:
 0.0      0.0      0.0      0.0     0.0   …    -0.0      -0.0      -0.0
 0.0     -0.0     -0.0     -0.0    -0.0        -0.0       0.0      -0.0
 0.0     -0.0     -0.0     -0.0    -0.0         0.0       0.0       0.0
 0.0      0.0      0.0      0.0     0.0        -0.0      -0.0      -0.0
 0.0     -0.0     -0.0      0.0    -0.0        -0.0      -0.0      -0.0
 0.0      0.0      0.0      0.0     0.0   …    -0.0      -0.0      -0.0
 0.0      0.0      0.0      0.0     0.0        -0.0      -0.0      -0.0
 0.0      0.0      0.0      0.0     0.0        -0.0      -0.0      -0.0
 0.0      0.0      0.0      0.0     0.0         0.0      -0.0      -0.0
 0.0     -0.0     -0.0      0.0    -0.0        -0.0      -0.0      -0.0
 0.0  11292.8  11644.5  11413.0     0.0   …   634.076   302.876  2096.71
 0.0      0.0      0.0   2627.5  3616.72     3616.72   3616.72   3616.72
 0.0      0.0      0.0      0.0     0.0         0.0      -0.0       0.0
 0.0      0.0      0.0      0.0     0

In [48]:
solutions.CHARGE

14×1001 Matrix{Float64}:
     0.0   -0.0     -0.0       -0.0    …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0    0.0      0.0        0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0    0.0      0.0        0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0   -0.0     -0.0       -0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0    0.0      0.0        0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0   -0.0     -0.0       -0.0    …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0   -0.0     -0.0       -0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0   -0.0     -0.0       -0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0   -0.0     -0.0       -0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
    -0.0    0.0      0.0        0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 13490.2    0.0      0.0    13490.2    …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0    0.0      0.0        0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0   -0.0     -0.0       -0.0       0.0  0.0  0.0  0.0  0

In [49]:
solutions.SOC

14×1001 Matrix{Float64}:
    -0.0      -0.0      -0.0      -0.0   …     -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0   …     -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
 57112.7   36580.3   15408.5    2077.2   …  61475.6   60924.9   57112.7
 37006.8   37006.8   37006.8   33357.4      47053.2   42030.0   37006.8
    -0.0      -0.0      -0.0      -0.0         -0.0      -0.0      -0.0
  4924.16   5020.63   5117.09   5213.56

In [58]:
solutions.GEN[:,2:10]

2×9 Matrix{Float64}:
   -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0    -0.0
 9021.1  9021.1  9021.1  9021.1  9021.1  9021.1  9021.1  9021.1  9021.1

In [61]:
solutions.CURTAIL[:,2:7]

3×6 Matrix{Float64}:
     0.0       0.0      0.0      0.0     0.0       0.0
  1151.64   1792.4      0.0      0.0     0.0       0.0
 32370.0   32370.0  22209.4  25650.8  6798.29  24526.5

In [62]:
halfhourlydemand

Row,hh,load
,Int64,Int64
1,30000,22635
2,30001,21974
3,30002,21685
4,30003,21147
5,30004,20772
6,30005,20869
7,30006,20808
8,30007,20619
9,30008,20521
